# Language Model with n-gram

Download BEST2010 corpus
(ref: https://aiforthai.in.th/corpus.php)

In [113]:
# get BEST2010.zip from Google drive
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1A9XA-h-SuTglmnti6m8S1saqozkYAf0x' -O BEST2010.zip
!unzip BEST2010.zip

--2023-09-07 16:31:42--  https://docs.google.com/uc?export=download&id=1A9XA-h-SuTglmnti6m8S1saqozkYAf0x
Resolving docs.google.com (docs.google.com)... 173.194.203.113, 173.194.203.101, 173.194.203.139, ...
Connecting to docs.google.com (docs.google.com)|173.194.203.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-90-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/3vq6pgurl0sd97cpek19jg6mq79sfdne/1694104275000/07034668329300256193/*/1A9XA-h-SuTglmnti6m8S1saqozkYAf0x?e=download&uuid=3ad9288f-12fd-4536-8760-c11e29547cd0 [following]
--2023-09-07 16:31:42--  https://doc-04-90-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/3vq6pgurl0sd97cpek19jg6mq79sfdne/1694104275000/07034668329300256193/*/1A9XA-h-SuTglmnti6m8S1saqozkYAf0x?e=download&uuid=3ad9288f-12fd-4536-8760-c11e29547cd0
Resolving doc-04-90-docs.googleusercontent.com (doc-04-90-docs.googleusercontent.com)... 74.125.197.132, 2607:f

In [247]:
#import necessary library: math, nltk, bigram, and collections.
import math
import nltk
import io
import random
from random import shuffle
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
import pandas as pd
random.seed(999)

In [248]:
#choose article domain as our dataset
best2010=[]
fp= io.open('BEST2010/article.txt','r',encoding='utf-8')

for i,line in enumerate(fp):
    best2010.append(line.strip()[:-1])
fp.close()
all_vocabulary =set()
total_word_count =0
for line in best2010:
    for word in line.split('|'):
        all_vocabulary.add(word)
        total_word_count+=1


In [249]:
#assumes each line is a sentence.
print ('Total sentences in BEST2010 news dataset :\t'+ str(len(best2010)))
print ('Total word counts in BEST2010 news dataset :\t'+ str(total_word_count))
print ('Total vocabulary in BEST2010 news dataset :\t'+ str(len(all_vocabulary)))

Total sentences in BEST2010 news dataset :	16990
Total word counts in BEST2010 news dataset :	1184905
Total vocabulary in BEST2010 news dataset :	25195


In [250]:
sentences = best2010
# The data is separated to train and test set with 70:30 ratio.
train = sentences[:int(len(sentences)*0.7)]
test = sentences[int(len(sentences)*0.7):]

#Training data
train_vocabulary = set()
train_word_count = 0
for line in train:
    for word in line.split('|'):
        train_vocabulary.add(word)
        train_word_count+=1
print ('Total sentences in BEST2010 news training dataset :\t'+ str(len(train)))
print ('Total word counts in BEST2010 news training dataset :\t'+ str(train_word_count))
print ('Total vocabuary in BEST2010 news training dataset :\t'+ str(len(train_vocabulary)))

# unknown word = 1/vocab_size
unk_value = 1/len(train_vocabulary)
print(unk_value)

Total sentences in BEST2010 news training dataset :	11893
Total word counts in BEST2010 news training dataset :	821549
Total vocabuary in BEST2010 news training dataset :	20669
4.8381634331607724e-05


# Unigram

In [251]:
def getUnigramModel(data):
    model = defaultdict(lambda: 0)
    word_count =0
    for sentence in data:
        sentence +=  u'|</s>' #for unigram model we can always ignore <s>, since p(w0=<s>)=1
        for w1 in sentence.split('|'):
            model[w1] += 1
            word_count+= 1
    for w1 in model:
        model[w1] = model[w1]/(word_count)
    return model

In [252]:
model = getUnigramModel(train)

In [253]:
def getLnValue(x):
    if x > 0.0:
        return math.log(x)
    else:
        return math.log(unk_value)

In [254]:
#problability of 'ประเทศ'
print(model[u'ประเทศ'])
print(getLnValue(model[u'ประเทศ']))

print(getLnValue(model[u'ประเทศมหาอำนาจ']))

#problability of 'จีน' 'เป็น' 'ประเทศ' 'มหาอำนาจ'
prob = getLnValue(model[u'จีน'])+getLnValue(model[u'เป็น'])+getLnValue(model[u'ประเทศ'])+getLnValue(model[u'มหาอำนาจ'])+getLnValue(model['</s>'])
print ('Problability of a sentence', math.exp(prob))

0.0021417207196181617
-6.146145698455239
-9.936390272211836
Problability of a sentence 3.9158991729834685e-15


**EX1:** Create perplexity function

In [255]:
def pp(log_prob, n):
  perplexity = math.exp(-log_prob/(n+1))
  return perplexity

**EX2:** Create a function for calculating perplexity of testing data




In [256]:
def calPp(data):
    model_word = defaultdict(lambda: 0)
    data_pp = {}
    word_count = 0
    for s in data:
      s += u'|</s>'

      for w in s.split('|'):
        model_word[w] += 1
        word_count += 1

    for s in data:
      s += u'|</s>'
      prob = 0
      n = 0

      for w in s.split('|'):
        prob += getLnValue(model_word[w]/(word_count))
        n += 1

      print(pp(prob, n))

calPp(test)

Streaming output truncated to the last 5000 lines.
149.3206451931928
201.44953977501197
410.53924052077997
166.76941438965335
354.72769327402654
351.55101854439397
217.808408442995
160.8430186989006
174.64648609051414
150.85074209353235
195.47993370081903
503.46164844727986
409.6481094132522
261.52884918251993
396.63750273850343
149.3206451931928
92.1250646188801
379.5477948678223
408.8832361782628
311.8518878934291
231.6091363563899
363.7023737221204
324.8007800977993
286.99472563491975
223.99446219206774
259.30347719591606
237.82363066387595
231.52705663899297
491.64389048853405
250.63149494929687
381.4755148703519
288.8591330063951
362.6141343740483
451.6445057056642
384.6503212055628
326.9617189976947
266.4909974447642
358.2550801123063
474.9758788377029
444.3146702792921
501.1244257574136
543.6481748890194
365.1090985149792
185.0132299005448
320.95972181008443
344.7594885910208
276.31976366309607
302.43887541749325
341.8438067837065
266.0000016256741
572.1174993341913
129.02045331

# Bigram

In [257]:
#example of nltk usage for bigram
sentence = 'I always search google for an answer .'

print('This is how nltk generate bigram.')
for w1,w2 in bigrams(sentence.split(), pad_right=True, pad_left=True):
    print (w1,w2)
print('None is used as a start and end of sentence symbol.')

This is how nltk generate bigram.
None I
I always
always search
search google
google for
for an
an answer
answer .
. None
None is used as a start and end of sentence symbol.


**Ex3:** Crete getBigramModel Function

In [258]:
def getBigramModel(data):
    model = defaultdict(lambda: defaultdict(lambda: 0))

    for s in data:
        s += u'|</s>'

        for w1,w2 in bigrams(s.split('|'), pad_right=True, pad_left=True):
          model[w1][w2] += 1

    for w1 in model:
        total_count = sum(model[w1].values())
        for w2 in model[w1]:
            model[w1][w2] = model[w1][w2] / total_count

    return model

**Ex4:** Create a function for calculating perplexity for bigram model of testing data

In [259]:
def pp_bigram(data, bi):
  log_prob = 0.0
  n = 0
  ls = []

  for sentence in data:
    sentence += u'|</s>'

    for w1,w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
        prob = bi[w1][w2]
        log_prob += getLnValue(prob)
        n += 1

    ls.append(pp(log_prob, n))
  return ls

pp_bigram(test, getBigramModel(test))

[26.129999670502695,
 22.87545430829416,
 18.520563396739483,
 22.536806673934436,
 21.461204499144145,
 18.122313282292225,
 19.147325994262857,
 22.188175699212582,
 21.036778978250847,
 26.372367921921,
 27.773336256568392,
 29.221595269607974,
 31.126515067347757,
 34.12679423898376,
 34.47317473077188,
 35.37083613821986,
 35.397744431266126,
 36.95078432993673,
 37.64498611720158,
 38.43223243709064,
 40.30641737439624,
 42.062061416124614,
 42.765660486656714,
 43.831069985524564,
 43.898047760574144,
 43.98810170981431,
 44.524744934723586,
 44.234484964851724,
 43.641720949308606,
 43.60837111168977,
 43.69740022380402,
 43.456720775054755,
 44.04505471583451,
 43.80471649632121,
 44.06730167416657,
 43.88243627084145,
 44.21307669033677,
 44.299796627888526,
 44.43032967410428,
 44.30047066875721,
 44.21127673582256,
 44.179346505182345,
 44.381049145329435,
 44.18866526302635,
 44.19750823883352,
 44.242418823556406,
 44.87767535218244,
 45.20180392588437,
 45.25047480583569

# Smoothing

Ex5: create getBigramWithInterpolation function

lambda value is 0.7 for bigram, 0.2 for unigram, and 0.1 for unknown word

In [260]:
def getBigramWithInterpolation(data, uni, bi):

    for s in data:
        s += u'|</s>'
        tokens = s.split('|')

    n = sum(uni.values())
    v_size = len(uni)

    def interpolate_prob(w1, w2):
        lambda_bigram = 0.7
        lambda_unigram = 0.2
        lambda_unknown = 0.1

        bigram_count = bi[w1][w2]
        unigram_count = uni[w2]

        if unigram_count == 0:
            unigram_count = 1

        interpolated_probability = (
            (lambda_bigram * bigram_count / unigram_count) +
            (lambda_unigram * unigram_count / n) +
            (lambda_unknown / v_size)
        )

        return interpolated_probability

    def sentence_prob(sentence):
        sentence += u'|</s>'
        tokens = sentence.split('|')
        probability = 1.0

        for w1, w2 in bigrams(tokens, pad_right=True, pad_left=True):
            probability *= interpolate_prob(w1, w2)

        return probability

    return sentence_prob

Ex6: Calculate Perplexity of testing data with bigraminterpolation

In [261]:
# Example usage:
cal_prob = getBigramWithInterpolation(train, getUnigramModel(train), getBigramModel(train))
shuffle(test)
sentence = test[0]

def cal_pp(prob, sentence):
    sentence += u'|</s>'
    tokens = sentence.split('|')
    sentence_length = len(tokens)

    perplexity = math.pow(prob, -1 / sentence_length)
    return perplexity

prob = cal_prob(sentence)
perplexity = cal_pp(prob, sentence)
print(f"Sentence: {sentence}")
print(f"Sentence prob: {prob}")
print(f"Perplexity: {perplexity}")

Sentence: ประชาธิปไตย|แบบ|มี|ส่วน|ร่วม|มอง|การ|เมือง|แยก|ออก|จาก|เศรษฐกิจ|,| |สังคม|,| |อุดมการณ์|,| |วัฒนธรรม| |และ|ปัจจัย|อื่น|ๆ| |และ|ไม่|ตระหนัก|ว่า|ความ|ไม่|เท่าเทียม|หรือ|ช่องว่าง|ทาง|อำนาจ|ที่|เกิด|ขึ้น|ใน|ปริมณฑล|อื่น|ๆ| |นั้น| |เป็น|อุปสรรค|ต่อ|การ|สร้าง|ประชาธิปไตย|แบบ|มี|ส่วน|ร่วม|ขึ้น|มา|ได้| |นอก|จาก|นั้น| |แนว|ความ|คิด|นี้|ยัง|มอง|ความ|สัมพันธ์|ระหว่าง|พลเมือง|ว่า|เป็น|เรื่อง|ของ|ความ|เท่าเทียม| |ทั้ง|ที่|การ|เมือง|ใน|ระบบ|เสรีประชาธิปไตย|เป็น|เรื่อง|ของ|พลเมือง|ที่|ถูก|ทำ|ให้|เหมือน|กัน| |(|homogeneity|)| |ไม่|ใช่|เป็น|ความ|สัมพันธ์|ระหว่าง|พลเมือง|ที่|เท่าเทียม|กัน| |(|equality|)
Sentence prob: 1.3330734171518386e+31
Perplexity: 0.5503338371109255


# Compare Perplexity

Ex7: แสดงตารางเปรียบเทียบค่า Perplexity แบบ
PP_Unigram,	PP_Bigram, PP_bigraminterpolation
ในการคำนวณข้อมูลใน testing data จำนวน 10 ประโยค

In [262]:
dataset = []
shuffle(test)
i = 0

for s in test:
  if i == 10:
    break
  dataset.append(s)
  i+=1

uni = getUnigramModel(train)
bi = getBigramModel(train)
bipol = getBigramWithInterpolation(train, getUnigramModel(train), getBigramModel(train))

results = []

for s in dataset:
    s += u'|</s>'
    tokens = s.split('|')
    n = len(tokens)
    uni_p = 0.0
    for w in tokens:
        uni_p += getLnValue(uni[w])
    perplexity = pp(uni_p, n)
    results.append(('unigram', perplexity))

for perplexity in pp_bigram(dataset, bi):
    results.append(('bigram', perplexity))

for s in dataset:
    prob = bipol(s)
    perplexity = cal_pp(prob, s)
    results.append(('bigram with interpolation', perplexity))

df = pd.DataFrame(results, columns=['Model', 'Perplexity'])

display(df)

,Model,Perplexity
0,unigram,371.774402
1,unigram,409.723259
2,unigram,344.856862
3,unigram,294.455222
4,unigram,472.939856
5,unigram,113.144768
6,unigram,636.942742
7,unigram,255.538812
8,unigram,319.640613
9,unigram,148.318847
